In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captioning/tensorflow2/model/1/model (1).h5
/kaggle/input/imgg/tensorflow1/model/1/model.h5
/kaggle/input/nlp_img_cap/tensorflow2/imgcapmodel/1/model.h5
/kaggle/input/unet/tensorflow2/unet_mn/1/unet_mn.h5
/kaggle/input/fcn/tensorflow2/fcn/1/fcn_model.h5
/kaggle/input/deep_lab/tensorflow2/deeplab/1/DeeplabV3_mobilenet.h5


In [5]:
!pip install streamlit -q

In [7]:
!pip install --upgrade keras tensorflow

import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.cluster import KMeans
import seaborn as sns
from tensorflow.keras.initializers import Orthogonal

custom_objects = {'Orthogonal': Orthogonal}

# Load your trained model
model = tf.keras.models.load_model("/kaggle/input/captioning/tensorflow2/model/1/model (1).h5", custom_objects=custom_objects)

In [ ]:
model.summary

In [ ]:
# Download ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

# Unzip the downloaded file
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
!chmod +x ngrok

In [ ]:
!./ngrok authtokens 2gTgesqVJ5fXr6FiY4N7R46L7hs_5FKChinrwKGCVpZw34uEA

In [ ]:
%%writefile app.py
import streamlit as st
st.title('Image Captioning App')

In [ ]:
import streamlit as st
import numpy as np
import os
from tqdm import tqdm
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
import pickle

# Load your trained captioning model
caption_model = load_model('path_to_your_caption_model.h5')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define maximum length of captions
max_length = 34  # Change this based on your model

# Load your word2idx and idx2word dictionaries
with open('word2idx.pkl', 'rb') as handle:
    word2idx = pickle.load(handle)

with open('idx2word.pkl', 'rb') as handle:
    idx2word = pickle.load(handle)

# Define the DenseNet201 model for feature extraction
img_size = 224
base_model = DenseNet201(weights='imagenet', include_top=False, pooling='avg')
fe = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

def predict_caption(model, feature, tokenizer, max_length, word2idx, idx2word):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
        y_pred = model.predict([np.array([feature]), np.array(sequence)])
        y_pred = np.argmax(y_pred)
        word = idx2word.get(y_pred)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return ' '.join(in_text.split()[1:-1])

# Streamlit app
st.title('Image Captioning App')

uploaded_file = st.file_uploader('Choose an image...', type='jpg')

if uploaded_file is not None:
    # Load and preprocess the image
    img = load_img(uploaded_file, target_size=(img_size, img_size))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Extract features using DenseNet201
    feature = fe.predict(img, verbose=0)
    
    # Generate the caption
    caption = predict_caption(caption_model, feature, tokenizer, max_length, word2idx, idx2word)
    
    # Display the image and the caption
    st.image(uploaded_file, caption='Uploaded Image', use_column_width=True)
    st.write('Generated Caption: ', caption)


In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501